# #6. Models 

0. [Motivation](#Motivation)
1. [Models](#Models)
1. [CRUD](#CRUD)
2. [N+1 problem](#N+1-problem)
3. [Object managers](#Object-managers)
4. [Raw](#Raw)
5. [LIMIT](#LIMIT)
6. [IN](#IN)


## Motivation

__Goal__

Познакомиться с основными составляющими уровня хранения, жизненным циклом данных и особенностями манипуляции ими с помощью ORM

__Homework__

Hasker!

## ORM

#### Models

* break apps with many models
    * 5 models per app preferably
* stick to fat models
    * instead of putting data-related code in views and templates use model's methods, properties, etc.
    * beware of god objects
        * use mixins and helpers (utils)
* normalize until it hurts, denormalize until it works.


__Model Inheritance__
* Abstract base classes
    * tables created only for derived classes
    * common fields in parent
    * cannot use parent abstract class

In [ ]:
#Model
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class user(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)

#Sessions
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

engine = create_engine('sqlite://example.db')
Session = sessionmaker(bind=engine)
session = Session()

#queries
users = session.query(User).filter(User.name=='Alice').all()

#annotations
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    user_id=Column(Integer, Foreignkey('users.id'))
    email_address = Column(String)
User.addresses - relationship("Address")


    


* Multi-table inheritance
    * tables created for parents and childs
    * can use parent and childs
    * can get child object from parent object
    * each query on a child table requires JOIN with ALL parent tables

In [ ]:
from django.db import models
class Animal(mocels.Model):
    name = models.CharField()
    age = models.IntegerField()

class Dog(Animal):
    ...


animal= Animal.objects.create(name="sdf", age=5)
dog = Dog.objects.create(name="sdf", age=3...)


dog = Dog.objects,get(id=1)
animal = dog.animal_ptr


animal = Animal.objects.get(id=1)
dog = animal.dog

* Proxy models
    * table created only for original model
    * cannot chage model behavior

In [ ]:
Base = declarative_base()

class User(Base):
    __tablename__='users'

    ...fields...
    def __repr__(self):
        return "1"

#proxy
class AdultUser(User):
    __mapper_args__ = {'polimorphic_identity': 'adult_user'}

    @property
    def is_adult(self):
        return sel.age >=18

    def __repr__(self):
        return "2"

#database with prosy models
engine = create_engine('sqlite://proxy_example.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

user1 = User(name="Alice", age=31)
session.add_all([user1,])

session.commit()
adults = session.query(AdultUser).filter(..).all()


#### CRUD

__Lazy evaluation__

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker, relationship

Base = declarative_base()

class User(Base):
    __tablename__='user'
    .. fields..

engine = create_engine('sqlite://proxy_example.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

user1 = User(name="Alice", age=31)
session.add_all([user1,])
session.commit()

query = ..any query..

for user in query:
    print(user.Name)

    

* A performance trade-off
* Calling a QuerySet method will usually clone the pre-existing QuerySet, apply the change, and return the new instance (doing a new query)
* Except for iteration, length/existence checks, which can re-use the existing QuerySet instance’s results cache without doing a new query

In [ ]:
queryset = MyModel.objects.filter(..)
queryset = queryset.exclude(name="sdf")

Do not evaluate query set unless you have to
* Defeats Django’s lazy-evaluation mechanism by dragging everything into memory.
* Does filtration or summarization in the application that is much more efficiently done in the database

In [ ]:
queryset = MyModel.object.filter(age=40)

print(len(queryset))
print(bool(queryset))

for obj in queryset:
    print(obj)

When QuerySets are evaluated
* Iteration
    * use update(), raw(), or custom iteration helper
* Slicing
    * generally, slicing a QuerySet returns a new QuerySet – it doesn’t evaluate the query
    * exception is if you use the “step” parameter of Python slice syntax
* repr()
* len()
    * use count()
* list()
* bool(), or, and, if
    * use exists()

* Once evaluated, a QuerySet holds on to its result set.
    * Reuse the same result set whenever possible!
* Remember that each filter you apply to a QuerySet creates a new query set.
    * That time can add up.
    * It can be faster to drop to raw SQL for super-complex queries.
* Don’t retrieve things you don’t need
    * use values() and values_list()
    * use defer() and only()

__Query expressions__

In [ ]:
F() Aggregate() Q() ExpressionWrapper() case when




In [ ]:
F

In [ ]:
from django.db.models import F
User.object.filter(age__lt=30).update(age=F('age') +1)

In [ ]:
Aggregation

In [ ]:
from django.db.models import Sum

total_age=User.objects.aggregate(Sum('age'))

In [ ]:
Q() condidtion

In [ ]:
from django.db.models import Q

users = User.objects.filter(Q(age_gt=30) | Q(name="Alice"))

In [ ]:
ExpressionWrapper()

In [ ]:
from django.db.models import ExpressionWrapper, F, IntegerField

age_difference = ExpressionWrapper(F('age') - F('years_odf_expierence'),output_field = Integer())
users = User.objects.annotate(age_diff = age_difference)

In [ ]:
Case When

In [ ]:
from django.db.models import Case, When, Value, IntegerField

users = User.objects.annotate(
    bonus = Case(
        When(age_gt=30, then Value(100)),
        ...
    )
)


#### N+1 problem

In [ ]:
class Author(models.Model):
    name=models.CharField()

class Book(models.Model):
    title=models.CharField()
    author = models.ForeignKey(Author, on_delete=models.CASCADE)


#N+1
books = Book.objects.all()
for book in books:
    pass

#solution
bools = Book.objects.select_related('author').all()


select_related
* used for foreign keys and one-to-one realtions
* results in a single more complex query with JOIN

prefetch_related
* used for many-to-many, many-to-one realtions
* does a separate lookup for each relationship
* does the ‘joining’ in Python
* object loaded in QuerySet cache (not lazy)
* in most cases will be implemented using an SQL query that uses the ‘IN’ operator

In [ ]:
class Author(models.Model):
    name=models.CharField()

class Book(models.Model):
    title=models.CharField()
    author = models.ForeignKey(Author, on_delete=models.CASCADE)


#N+1
books = Book.objects.all()
for book in books:
    pass

#solution
bools = Book.objects.prefetch_related('author').all()

#### Object managers

In [ ]:
class Author(models.Model):
    name=models.CharField()

class Book(models.Model):
    title=models.CharField()
    author = models.ForeignKey(Author, on_delete=models.CASCADE)


bools = Book.objects.prefetch_related('author').all()

objects - это менеджер обьектов



In [ ]:
from django.db import models

class Book(models.Model):
    title=models.CharField()

    class BookManager(models.Manager):
        def publisched_after(self, year):
            return self.filter(published_date__year__gt=year)
    objects = models.Manager()
    custom_manager=BookManager()

recent_books = Book.custom_manager.publisched_after(2020)


Adding new object manager
* when using model inheritance children of ABC recieve their parent's manager and children of CBC do not
* first manager applied to a model class Django treats as default
    * put obejcts = models.Manager() first

* Overriding get_queryset() will affect all queries made through that manager
* Usually best to keep a vanilla manager around so you can access everything normally

New manager v.s. New Query set

In [ ]:
from django.db import models

class Book(models.Model):
    title=models.CharField()

    class BookManager(models.Manager):
        def get_queryset(self):
            return super().get_queryset().filter(...)
    books=BookManager()

recent_books = Book.objects.all()

#### Raw

In [ ]:
from django.db import models

class Book(models.Model):
    title=models.CharField()


books = Book.objects.raw('SELECT ...')

for book in books:
    pass

#### LIMIT

* qs[:x] adds a LIMIT clause to the SQL.
* Remember that LIMIT isn’t really useful without a sort.
    * database has to sort the entire result set before applying the LIMIT.
* an index on the sort key is a good idea
* qs[x:y] does an OFFSET x LIMIT y-x.
* OFFSET has to consider and toss every object from 1 to x-1.
* Very large OFFSETs are extremely inefficient.
* Much better to use queries on indexed columns instead.
* For pagination, consider strongly limiting how deep it can go.

In [ ]:
from django.db import models

class Book(models.Model):
    title=models.CharField()


books = Book.objects.raw('SELECT ... DESC LIMIT %s', [5])

#### IN

In [ ]:
Thing.objects.filter(id__in=my_little_list)

In [1]:
 SELECT ""stuff_thing"".""id"", ""stuff_thing"".""thing1"", ""stuff_thing"".""thing2"" FROM ""stuff_thing"" WHERE
    ""stuff_thing"".""id"" IN (3702, 3705, 3708, 3711, 3714, 3717, 3720, 3723, 3726, 3729, 3732, 3735, 3738, 3741, 3744, 3747, 3750,
    3753, 3756, 3759, 3762, 3765, 3768, 3771, 3774, 3777, 3780, 3783, 3786, 3789, 3792, 3795, 3798, 3801, 3804, 3807, 3810, 3813, 3816,
    3819, 3822, 3825, 3828, 3831, 3834, 3837, 3840, 3843, 3846, 3849, 3852, 3855, 3858, 3861, 3864, 3867, 3870, 3873, 3876, 3879, 3882,
    3885, 3888, 3891, 3894, 3897, 3900, 3903, 3906, 3909, 3912, 3915, 3918, 3921, 3924, 3927, 3930, 3933, 3936, 3939, 3942, 3945, 3948,
    3951, 3954, 3957, 3960, 3963, 3966, 3969, 3972, 3975, 3978, 3981, 3984, 3987, 3990, 3993, 3996, 3999, 4002, 4005, 4008, 4011, 4014,
    4017, 4020, 4023, 4026, 4029, 4032, 4035, 4038, 4041, 4044, 4047, 4050, 4053, 4056, 4059, 4062, 4065, 4068, 4071, 4074, 4077, 4080,
    4083, 4086, 4089, 4092, 4095, 4098, 4101, 4104, 4107, 4110, 4113, 4116, 4119, 4122, 4125, 4128, 4131, 4134, 4137, 4140, 4143, 4146,
    4149, 4152, 4155, 4158, 4161, 4164, 4167, 4170, 4173, 4176, 4179, 4182, 4185, 4188, 4191, 4194, 4197, 4200, 4203, 4206, 4209, 4212,
    4215, 4218, 4221, 4224, 4227, 4230, 4233, 4236, 4239, 4242, 4245, 4248, 4251, 4254, 4257, 4260, 4263, 4266, 4269, 4272, 4275, 4278,
    4281, 4284, 4287, 4290, 4293, 4296, 4299, 4302, 4305, 4308, 4311, 4314, 4317, 4320, 4323, 4326, 4329, 4332, 4335, 4338, 4341, 4344,
    4347, 4350, 4353, 4356, 4359, 4362, 4365, 4368, 4371, 4374, 4377, 4380, 4383, 4386, 4389, 4392, 4395, 4398, 4401, 4404, 4407, 4410,
    4413, 4416, 4419, 4422, 4425, 4428, 4431, 4434, 4437, 4440, 4443, 4446, 4449, 4452, 4455, 4458, 4461, 4464, 4467, 4470, 4473, 4476,
    4479, 4482, 4485, 4488, 4491, 4494, 4497, 4500, 4503, 4506, 4509, 4512, 4515, 4518, 4521, 4524, 4527, 4530, 4533, 4536, 4539, 4542,
    4545, 4548, 4551, 4554, 4557, 4560, 4563, 4566, 4569, 4572, 4575, 4578, 4581, 4584, 4587, 4590, 4593, 4596, 4599, 4602, 4605, 4608,
    4611, 4614, 4617, 4620, 4623, 4626, 4629, 4632, 4635, 4638, 4641, 4644, 4647, 4650, 4653, 4656, 4659, 4662, 4665, 4668, 4671, 4674,
    4677, 4680, 4683, 4686, 4689, 4692, 4695, 4698, 4701, 4704, 4707, 4710, 4713, 4716, 4719, 4722, 4725, 4728, 4731, 4734, 4737, 4740,
    4743, 4746, 4749, 4752, 4755, 4758, 4761, 4764, 4767, 4770, 4773, 4776, 4779, 4782, 4785, 4788, 4791, 4794, 4797, 4800, 4803, 4806,
    4809, 4812, 4815, 4818, 4821, 4824, 4827, 4830, 4833, 4836, 4839, 4842, 4845, 4848, 4851, 4854, 4857, 4860, 4863, 4866, 4869, 4872,
    4875, 4878, 4881, 4884, 4887, 4890, 4893, 4896, 4899, 4902, 4905, 4908, 4911, 4914, 4917, 4920, 4923, 4926, 4929, 4932, 4935, 4938,
    4941, 4944, 4947, 4950, 4953, 4956, 4959, 4962, 4965, 4968, 4971, 4974, 4977, 4980, 4983, 4986, 4989, 4992, 4995, 4998, 5001, 5004,
    5007, 5010, 5013, 5016, 5019, 5022, 5025, 5028, 5031, 5034, 5037, 5040, 5043, 5046, 5049, 5052, 5055, 5058, 5061, 5064, 5067, 5070,
    5073, 5076, 5079, 5082, 5085, 5088, 5091, 5094, 5097, 5100, 5103, 5106, 5109, 5112, 5115, 5118, 5121, 5124, 5127, 5130, 5133, 5136,
    5139, 5142, 5145, 5148, 5151, 5154, 5157, 5160, 5163, 5166, 5169, 5172, 5175, 5178, 5181, 5184, 5187, 5190, 5193, 5196, 5199, 5202,
    5205, 5208, 5211, 5214, 5217, 5220, 5223, 5226, 5229, 5232, 5235, 5238, 5241, 5244, 5247, 5250, 5253, 5256, 5259, 5262, 5265, 5268,
    5271, 5274, 5277, 5280, 5283, 5286, 5289, 5292, 5295, 5298, 5301, 5304, 5307, 5310, 5313, 5316, 5319, 5322, 5325, 5328, 5331, 5334,
    5337, 5340, 5343, 5346, 5349, 5352, 5355, 5358, 5361, 5364, 5367, 5370, 5373, 5376, 5379, 5382, 5385, 5388, 5391, 5394, 5397, 5400,
    5403, 5406, 5409, 5412, 5415, 5418, 5421, 5424, 5427, 5430, 5433, 5436, 5439, 5442, 5445, 5448, 5451, 5454, 5457, 5460, 5463, 5466,
    5469, 5472, 5475, 5478, 5481, 5484, 5487, 5490, 5493, 5496, 5499, 5502, 5505, 5508, 5511, 5514, 5517, 5520, 5523, 5526, 5529, 5532,
    5535, 5538, 5541, 5544, 5547, 5550, 5553, 5556, 5559, 5562, 5565, 5568, 5571, 5574, 5577, 5580, 5583, 5586, 5589, 5592, 5595, 5598,
    5601, 5604, 5607, 5610, 5613, 5616, 5619, 5622, 5625, 5628, 5631, 5634, 5637, 5640, 5643, 5646, 5649, 5652, 5655, 5658, 5661, 5664,
    5667, 5670, 5673, 5676, 5679, 5682, 5685, 5688, 5691, 5694, 5697, 5700, 5703, 5706, 5709, 5712, 5715, 5718, 5721, 5724, 5727, 5730,
    5733, 5736, 5739, 5742, 5745, 5748, 5751, 5754, 5757, 5760, 5763, 5766, 5769, 5772, 5775, 5778, 5781, 5784, 5787, 5790, 5793, 5796,
    5799, 5802, 5805, 5808, 5811, 5814, 5817, 5820, 5823, 5826, 5829, 5832, 5835, 5838, 5841, 5844, 5847, 5850, 5853, 5856, 5859, 5862,
    5865, 5868, 5871, 5874, 5877, 5880, 5883, 5886, 5889, 5892, 5895, 5898, 5901, 5904, 5907, 5910, 5913, 5916, 5919, 5922, 5925, 5928,

SyntaxError: invalid syntax (153761523.py, line 1)

* large IN's are bad
* Very expensive for the database to parse
* Very expensive for the database to execute.
* If there are potentially more than 10-15 items in the list, rework the IN as a JOIN against whatever the source of the keys is.

In [ ]:

with ids AS (
    select 1 As id UNION ALL
    SELECT 2 UNION ALL ...
)
select b.*
from books b
join ids 1 on b.id=i.id

### References

* https://www.youtube.com/playlist?list=PLrCZzMib1e9r6c-j8aW1JuETSyCBp9iAg
* https://lukeplant.me.uk/blog/posts/avoid-django-genericforeignkey/
* https://docs.djangoproject.com/en/4.2/topics/db/optimization/
* http://blog.kevinastone.com/django-model-behaviors.html
* https://medium.com/eshares-blog/supercharging-django-productivity-at-eshares-8dbf9042825e